# Day 10: Smoke Basin

In [1]:
from pathlib import Path
from collections import Counter, deque
from functools import reduce

from aoc2021.util import read_as_list

## Puzzle input data

In [2]:
# Test data.
tdata = [
    '[({(<(())[]>[[{[]{<()<>>',
    '[(()[<>])]({[<{<<[]>>(',
    '{([(<{}[<>[]}>{[]{[(<()>',
    '(((({<>}<{<{<>}{[]{[]{}',
    '[[<[([]))<([[{}[[()]]]',
    '[{[{({}]{}}([{[{{{}}([]',
    '{<[[]]>}<{[{[{[]{()[[[]',
    '[<(<(<(<{}))><([]([]()',
    '<{([([[(<>()){}]>(<<{{',
    '<{([{{}}[<[[[<>{}]]]>[]]',
]

# Input data.
data = read_as_list(Path('./day10-input.txt'), func=str.rstrip)
data[:5]

['{{<{{{{([{[([[()<>]{<>{}}]<([]())(()<>)>)((({}())[()[]])<<[][]>[{}[]]>)]{{(<{}<>>{<><>}]([<>[]]<',
 '[(<{{[{(<({{<<[]()><<>{}>>([<>[]]{<><>})}})>)}]}}>[{(<{({[{[[({}())((){})]({{}[]})]<<[<>{}]([][])>({<>()}',
 '(({<{[{({(([[([]())({}())]]({[[]{}]([][]))<((){})<{}<>>>))[(([<>[]]<[]>)(([]{}){{}{}}))])})[({<[{',
 '([{{[([<({<<<([]())[()[]]>{<()[]>[[]()]}>[{<[]{}><[]>>{<<>()>{[]()}}]>[[[[[]{}]([]<>)]<{<>{}}',
 '[[((<({<(<{<<{{}()}{[][]}>[((){})]>}>{((<({}<>)<{}()>>[[<>()]])<<<[][]><<>[]>>{<{}[]>(<>())}>)<{[[{']

## Puzzle answers
### Part 1

In [3]:
Input = list[list[str]]


def matching(c: str) -> str:
    match c:
        case '(': return ')'
        case ')': return '('
        case '[': return ']'
        case ']': return '['
        case '{': return '}'
        case '}': return '{'
        case '<': return '>'
        case '>': return '<'
        case _: raise Exception('invalid character {c}')


def parse_line(line: str) -> list[str]:
    """Return ([<first illegal character>], <stack remaining>)."""
    openings = {'(', '[', '{', '<'}
    closings = {')', ']', '}', '>'}
    stack = deque()
    for c in line:
        if c in openings:
            stack.append(c)
        elif c in closings:
            if matching(c) != stack.pop():
                return [c], ''.join(stack)
        else:
            raise Exception('invalid character {c}')
    return [], ''.join(stack)


def corrupt_errors(data: Input) -> list[str]:
    return [c for cs,_ in map(parse_line, data) for c in cs]


def corrupt_errscore(errs: list[str]) -> int:
    points = {')': 3, ']': 57, '}': 1197, '>': 25137}
    return sum(cnt*points[e] for e,cnt in Counter(errs).items())


assert parse_line('[(()[<>])]({[<{<<[]>>(')[0] == []
assert parse_line('{([(<{}[<>[]}>{[]{[(<()>')[0] == ['}']
assert len(corrupt_errors(tdata)) == 5
assert corrupt_errors(tdata) == ['}',')',']',')','>']
assert corrupt_errscore(corrupt_errors(tdata)) == 26397

In [4]:
n = corrupt_errscore(corrupt_errors(data))
print(f'The total syntax error score for corruption errors: {n}')

The total syntax error score for corruption errors: 362271


### Part 2

In [5]:
def autocomplete(stack: str) -> str:
    return ''.join(map(matching, reversed(stack)))


def points(c: str) -> int:
    match c:
        case ')': return 1
        case ']': return 2
        case '}': return 3
        case '>': return 4
        case _: raise Exception('invalid character {c}')


def autocomplete_score(css: list[str]) -> int:
    return reduce(lambda x,y: 5*x + y, map(points, css), 0)


def autocompletions(data: Input) -> list[str]:
    return [autocomplete(st) for cs,st in map(parse_line, data) if not cs]


def median(xs: list[int]) -> int:
    return sorted(xs)[len(xs) // 2]


def mid_autocomp_score(data: Input) -> int:
    return median(list(map(autocomplete_score, autocompletions(data))))


assert parse_line('[{}<([])()>]')[1] == ''
assert autocomplete(parse_line('[({(<(())[]>[[{[]{<()<>>')[1]) == '}}]])})]'
assert len(autocompletions(tdata)) == 5
assert autocomplete_score(')}>]})') == 5566
assert median([5,7,0,2,1]) == 2
assert mid_autocomp_score(tdata) == 288957

In [6]:
n = mid_autocomp_score(data)
print(f'The middle autocompletion score: {n}')

The middle autocompletion score: 1698395182
